# Create Mapper Model

In [1]:
import sys
import re
import fileinput
import sys
import os
import math
from pathlib import Path

import collections
import numpy as np
import pandas as pd

import time
from scipy.spatial.transform import Rotation
from scipy.spatial.distance import cdist
from scipy.stats import zscore
import statistics
import open3d as o3d

## Input Paths

In [2]:
#registered 5 central camera poses into right camera model now must align center and right model
path_mapped = '/home/rapires/Documents/Exp_Thesis/OTANIEMI/otaniemi_25_04_2021/Models/mapper_Model/model1_cam_right_10fps/new_sparse'
path_central = '/home/rapires/Documents/Exp_Thesis/OTANIEMI/otaniemi_25_04_2021/Models/model1_cam_center_10fps/sparse/0'

cart_mapped = '/home/rapires/Documents/Exp_Thesis/python-scripts/data/cam_aligned_rightcenter10fps'
cart_central = '/home/rapires/Documents/Exp_Thesis/python-scripts/data/camcentral_10fps'

path_cart_mapped = Path(cart_mapped, 'img_cartesian.txt')
path_cart_central = Path(cart_central, 'img_cartesian.txt')

path_img_mapped = Path(path_mapped, 'images.txt')
path_img_central = Path(path_central, 'images.txt')

path_3d_mapped = Path(path_mapped, 'points3D.txt')
path_3d_central = Path(path_central, 'points3D.txt')
#colmap_data = extract_data_colmap(imagestxt)

### extract data methods

In [4]:
def extract_data_3dpoints(filepath):
    """
    Extracts data from a file generated by Colmap called points3D.txt,
    changes collors of the points for visualization.
    Returns:
    [dict] pointid: its 3d coordinates list x,y,z
    """
    line_count = 0
    points_list = {}
    
    with open(filepath, "r") as f:
        next(f)
        next(f)
        next(f)
        all_lines = f.readlines()

        for idx, line in enumerate(all_lines):
            
            point_id, x, y, z, r, g, b, _  = line.split()[:8]
            track = line.split()[8:]

            single_track = []
            all_tracks = []
            for idx2, line2 in enumerate(track):
                single_track.append(line2)
                if (idx2+1) % 2 == 0:
                    all_tracks.append(single_track)
                    single_track = []

            #print(all_tracks)
            if len(all_tracks) > 3:
                points_list[point_id] = [x, y, z, r, g, b, all_tracks]
            #break
    od = collections.OrderedDict(sorted(points_list.items()))
    points_list = dict(od)
    return points_list


def extract_data_colmap(filepath):
    """
    This function extracts data from a file generated by the Colmap, called images.txt
    track[] as imageid, point2did

    Returns: 
    [Dictionary] image-name: image id, list of 2d keypoints and 3d points that are observed by the keypoints
    """
    data = {}
    with open(filepath, "r") as f:
        next(f)  # skip the 3 header lines
        next(f)
        next(f)
        next(f)
        for idx, line in enumerate(f):
            if idx % 2 == 0:
                image_id, qw, qx, qy, qz, tx, ty, tz, _, image_name = line.split()
                #image_name = images_path + image_name
            else:
                info = line.split()

                keyframe = []
                all_keyframes = []

                for idx2, line2 in enumerate(info):

                    keyframe.append(line2)
                    if (idx2+1) % 3 == 0:
                        '''if line2 != '-1':
                            outer_list.append(inner_list)'''
                        all_keyframes.append(keyframe)
                        keyframe = []

                data[image_id] = [image_name, all_keyframes, qw, qx, qy, qz, tx, ty, tz]
            #break
    od = collections.OrderedDict(sorted(data.items()))
    data = dict(od)
    return data

def extract_img_cartesian(filepath):
    """Returns dict with image cartesian coordinates"""
    #od = collections.OrderedDict(sorted(img_model.items())) Order dictionary of images
    t = dict()
    with open(filepath, 'r') as f:
        lines = f.readlines()
    for line in lines:
        key, values = tuple(line.split(':'))
        #print(key)
        key, img_id = tuple(key.split(', '))
        values = list(str(re.sub(r'[\[\]\n]','',values)).split(' '))
        values = [float(i) for i in values if len(i) != 0]
        values.append(img_id)
        t[key] = values
    od = collections.OrderedDict(sorted(t.items()))
    t = dict(od)
    return t


### Scale Methods

In [5]:
def mean_increment(cart_mapped, cart_central, n = 5):
    # estimates scale_bigmodel / scale_small model relation for the transformation based on camera poses
    # n specifies camera index jump, n = 1 key 'center_1000' to 'center_1001', n = 2, key 'center_1000' to 'center_1002'
    dif_mapped = get_diff(cart_mapped, n)
    dif_central = get_diff(cart_central, n)
    
    dif_mapped = sum(dif_mapped)/len(dif_mapped)
    dif_central = sum(dif_central)/len(dif_central)
    
    increment = dif_mapped / dif_central
    return increment
    
def get_diff(cart_img, n = 5):
    """returns list of the scalar differences among consecutive images"""
    img_dif = list()
    item = iter(cart_img.items())
    key, value = next(item)
    
    for i in range(n-1): #images keys '<name>_<index>.jpg'
        name, index = tuple(key.rsplit('_',1))
        index = int(str(index.split('.')[0]))
        
        dif_v = np.array(cart_img[f'{name}_{str(index+i+1)}.jpg'][:3]) - np.array(cart_img[f'{name}_{str(index+i)}.jpg'][:3])
        img_dif.append(np.linalg.norm(dif_v))
        
        #print(f'center_{str(index+i+1)} minus center_{str(index+i)} equals {np.linalg.norm(dif_v)}')            
    return img_dif    

### Tranformation methods 

In [6]:
def rotation_matrix(unit_1, unit_2):
    """ROTATION MATRIX"""
    v = np.cross(unit_1, unit_2)
    s = np.linalg.norm(v)
    c = np.dot(unit_1, unit_2)

    I = np.identity(3)
    m = f'0 {float(-v[2])} {float(v[1])}; {float(v[2])} 0 {float(-v[0])}; {float(-v[1])} {float(v[0])} 0'
    V = np.matrix(m)

    R = I + V + (V**2)*(1/(1+c))

    return R

def get_uvectors(cart_img, name, n=150):
    """get avg unit vector"""
    uvectors = list()
    item = iter(cart_img.items())
    key, value = next(item)
    ref = cart_img[f'{name}_1000.jpg'][:3]
    
    start, index = tuple(key.rsplit('_',1))
    index = int(str(index.split('.')[0]))
    
    for i in range(1,n):
        v = np.array(cart_img[f'{name}_{index+i}.jpg'][:3]) - np.array(ref)
        uv = v/np.linalg.norm(v)
        uvectors.append(uv)
    
    mat = np.matrix(uvectors)
    meanv = mat.mean(0)
    meanv = np.asarray(meanv[0])[0]
    mean_uv = meanv / np.linalg.norm(meanv)
    return mean_uv

In [7]:
def match_cartesian(cart_central, cart_mapped, n):
    """match central cameras in both models, perform necessary scaling and rotations"""
    scale = mean_increment(cart_mapped, cart_central, n)
    #mapped model is slightly (5%) bigger than the central only based on 5 images might not need scaling
    #print(scale)
    mapped_uv = get_uvectors(cart_mapped,'right', n=5)
    central_uv = get_uvectors(cart_central,'center', n=5)
    
    R = rotation_matrix(central_uv, mapped_uv)
    
    #print(mapped_uv), print(central_uv)
    return scale, R

In [8]:
def get_pcd(cart_img, points3d):
    #with open(filepath) as f:
    points = []
    colors = []
    for key, value in cart_img.items():
        if 'center' in key:
            #color = '0.788,0.301,0.878' #purple
            color = [0.788, 0.301, 0.878]
            colors.append(color)
        else:
            #color = '0.023, 0.698, 0.078' #green
            color = [0.023, 0.698, 0.078]
            colors.append(color)
        point = [value[0], value[1], value[2]]
        points.append(point)
        #line = f'{value[0]}, {value[1]}, {value[2]}, {color}\n' 
    for key, value in points3d.items(): 
        #color = '0.458, 0.458, 0.458\n' #grey
        #line = f'{value[0]}, {value[1]}, {value[2]}, {color}\n' 
        color = [0.458, 0.458, 0.458]
        point = [value[0], value[1], value[2]]
        points.append(point)
        colors.append(color)

    pcd = o3d.geometry.PointCloud() 
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

In [9]:
def find_plane(pcd):
    """find 8 points in plane of the COLMAP model which delimit the model"""
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.3,
                                             ransac_n=10,
                                             num_iterations=10000)
    [a, b, c, d] = plane_model
    #print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    #print(f'Normal Vector to the plane: {a: .2f}i + {b: .2f}j + {c: .2f}k')
    normal = np.array([a,b,c])
    unormal = normal/np.linalg.norm(normal)
    #####
    return unormal, plane_model

In [10]:
def get_angle(normal1, normal2):
    """get angle between two normal vectors from models aligned in same direction"""
    dot_product = np.dot(normal1, normal2)
    angle = np.arccos(dot_product)
    return angle
    

## Main (Registration)

In [30]:
#1st extract data, and find spatial relations

points3d_mapped = extract_data_3dpoints(path_3d_mapped)
points3d_central = extract_data_3dpoints(path_3d_central)

images_mapped = extract_data_colmap(path_img_mapped)
images_central = extract_data_colmap(path_img_central)

cart_mapped = extract_img_cartesian(path_cart_mapped)
cart_central = extract_img_cartesian(path_cart_central)

pcd_center = get_pcd(cart_central, points3d_central)
pcd_mapped = get_pcd(cart_mapped, points3d_mapped)

scale = mean_increment(cart_mapped, cart_central, n=5)

mapped_uv = get_uvectors(cart_mapped,'right', n=100)
central_uv = get_uvectors(cart_central,'center', n=100) 


R = rotation_matrix(mapped_uv, np.array([0,1,0]))
pcd_mapped.rotate(R)
R = rotation_matrix(central_uv, np.array([0,1,0]))
pcd_center.rotate(R)

#2nd write to open3d point cloud

cnormal, cplane = find_plane(pcd_center)
mnormal, mplane = find_plane(pcd_mapped)

angle = get_angle(cnormal, mnormal)
R = o3d.geometry.get_rotation_matrix_from_xyz((0, angle, 0))
pcd_center.rotate(R)

a = np.asarray(pcd_center.points)
b = np.asarray(pcd_mapped.points)
translation = b[0] - a[0]

#print(a)
#print(b)
print(translation)
pcd_center.translate(translation)
o3d.visualization.draw_geometries([pcd_mapped, pcd_center])

[ 0.48524634  1.16190899 -1.22113382]


### Registration Output Paths

In [18]:
variable = tuple((pcd_mapped, pcd_center, points3d_mapped, points3d_central, images_mapped, images_central, cart_mapped, cart_central))

In [18]:
output_path = '/home/rapires/Documents/Exp_Thesis/python-scripts/data/cam_aligned_rightcenter10fps/1_Registration'
output_model = Path(output_path, 'entire_model.txt')
output_images_central = Path(output_path, 'images_central.txt')
output_images_mapped = Path(output_path, 'images_mapped.txt')
output_points_central = Path(output_path, 'points3d_central.txt')
output_points_mapped = Path(output_path, 'points3d_mapped.txt')
outputc_cartesian = Path(output_path, 'imgc_cartesian.txt')
outputm_cartesian = Path(output_path, 'imgm_cartesian.txt')
create_files = f'touch {output_model} {output_images_central} {output_images_mapped} {output_points_central} {output_points_mapped} {output_cartesian}'
os.system(create_files)

0

### Write to files

In [31]:
def write_model(variable, output_model, output_images_mapped, output_images_central, output_points_central, output_points_mapped):
    """"""
    pcd_mapped, pcd_center, points3d_mapped, points3d_center, images_mapped, images_center, cart_mapped, cart_central = variable
    pmapped = np.asarray(pcd_mapped.points)
    pcenter = np.asarray(pcd_center.points)
    cmapped = np.asarray(pcd_mapped.colors)
    ccenter = np.asarray(pcd_center.colors)
    
    pimg_mapped, pp3d_mapped, pimg_center, pp3d_center = [], [], [], []
    
    with open(output_model, 'w') as f:
        for i in range(len(pmapped)):
            line = f'{str(pmapped[i][0])}, {str(pmapped[i][1])}, {str(pmapped[i][2])}, '+\
            f'{str(cmapped[i][0])}, {str(cmapped[i][1])}, {str(cmapped[i][2])}\n'
            f.write(line)
            if '0.458' not in str(cmapped[i]): # if not a 3d point, it's an image
                pimg_mapped.append(pmapped[i])
            else:
                pp3d_mapped.append(pmapped[i])
        
        for i in range(len(pcenter)):
            line = f'{str(pcenter[i][0])}, {str(pcenter[i][1])}, {str(pcenter[i][2])}, '+\
            f'{str(ccenter[i][0])}, {str(ccenter[i][1])}, {str(ccenter[i][2])}\n'
            f.write(line)
            if '0.458' not in str(ccenter[i]): # if not a 3d point, it's an image
                pimg_center.append(pcenter[i])
            else:
                pp3d_center.append(pcenter[i])
        
    for idx, (key, value) in enumerate(cart_mapped.items()):
        cart_mapped[key] = np.append(pimg_mapped[idx], value[3])
    
    for idx, (key, value) in enumerate(cart_central.items()):
        cart_central[key] = np.append(pimg_center[idx], value[3])
        
    write_cartesian(output_cartesian, cart_central, cart_mapped)
    write_images(output_images_mapped, cart_mapped, images_mapped)
    write_images(output_images_central, cart_central, images_central)
    write_points(output_points_mapped, pp3d_mapped, points3d_mapped)
    write_points(output_points_central, pp3d_center, points3d_central)
    
def write_cartesian(output_cartesian, cart1, cart2):
    """write to cartesian file"""
    with open(output_cartesian,'w') as f:
        for key, value in cart1.items():
            line = f'{key}, {value[3]}: {value[:3]}'
            line = re.sub(r"'", '', line) + '\n'
            f.write(line)
        for key, value in cart2.items():
            line = f'{key}, {value[3]}: {value[:3]}'
            line = re.sub(r"'", '', line) + '\n'
            f.write(line)

def write_points(output_points, pp3d, points3d):
    """ write in points3d file"""
    with open(output_points, 'w') as f:
        start_line = f'# 3D point list with one line of data per point:\n'+\
    f'# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)\n'+\
    f'# Number of points: {len(pp3d)}, mean track length: Nan\n'
        f.write(start_line)
        
        for idx, (key, value) in enumerate(points3d.items()):
            line = f'{key} {pp3d[idx]} {value[3:6]}, error, {value[6]}'
            line = re.sub(r"[\[\]\'\',]", '', line) + '\n'
            f.write(line)
    
def write_images(output_images, cart, images):
    """ write in image.txt file"""
    with open(output_images,'w') as f:
        start_line = f'# Image list with two lines of data per image:\n'+\
        '#   IMAGE_ID, X, Y, Z, NAME\n'+\
        '#   POINTS2D[] as (X, Y, POINT3D_ID)\n'+\
        f'#   Number of images: {len(images)}\n'
        f.write(start_line)

        for key, value in cart.items():
            img_key = value[3]
            line = f'{img_key} {value[:3]} {key}'
            nextline = f'{images[img_key][1]}'
            line = re.sub(r"[\[\]\'\',]", '', line) + '\n'
            nextline = re.sub(r"[\[\]\'\',]", '', nextline) + '\n'
            f.write(line)
            f.write(nextline)
    
    
write_model(variable, output_model, output_images_mapped, output_images_central, output_points_central, output_points_mapped)

## Transform

In [23]:
path = '/home/rapires/Documents/Exp_Thesis/OTANIEMI/otaniemi_model/back_iphone_seq'
original_path_3d = path +"/sparse/0/points3D.txt" #from the colmap model
img_cartesian = 'data/big_otaniemi_model/img_cartesian.txt'

points3d_bm = extract_data_3dpoints(original_path_3d)
images_bm = extract_img_cartesian(img_cartesian)

In [25]:
points3d_c = extract_data_3dpoints(output_points_central)
points3d_m = extract_data_3dpoints(output_points_mapped)

images_c = extract_img_cartesian(output_cartesian)
images_m = extract_img_cartesian(output_cartesian)

rm_keyc = [key for key, value in images_c.items() if 'right' in key]
for key in rm_keyc:
    images_c.pop(key, None)        
rm_keym = [key for key, value in images_m.items() if 'center' in key]
for key in rm_keym:
    images_m.pop(key, None)        
#my_dict.pop('key', None)

### Transform Output Paths

In [29]:
scale = mean_increment(images_bm, images_c, 100) #the mean increment will have to be the same


KeyError: 'center_10001.jpg'

### Output Transformation

In [ ]:
path = '/home/rapires/Documents/Exp_Thesis/python-scripts/data/cam_aligned_rightcenter10fps/2_Transformation'
pathc = Path(path, 'central')
pathm = Path(path, 'mapped')
outc_timages, outc_tpoints, outc_tmodel = Path(pathc, 'images.txt'), Path(pathc, 'points3D.txt'), Path(pathc, 't_model.txt')
outm_timages, outm_tpoints, outm_tmodel = Path(pathm, 'images.txt'), Path(pathm, 'points3D.txt'), Path(pathm, 't_model.txt')
create_files = f'touch {outc_timages} {outc_tpoints} {outc_tmodel} {outm_timages} {outm_tpoints} {outm_tmodel}'
os.system(create_files)

### Testing Code

In [34]:
item = iter(cart_central.items())
key, value = next(item)
print(key)
print(value)

AttributeError: 'str' object has no attribute 'items'

In [17]:
print(angle)

0.054981296810657825


In [21]:
item = iter(points3d_mapped.items())
key, value = next(item)
print(key)
print(value)
#print(list(range(5)))
#print(len(images_central.keys()))

10000
['-1.0295790855357168', '-0.06436695077816014', '1.0834355799518396', '26', '27', '22', [['55', '3038'], ['57', '2829'], ['56', '3051'], ['58', '2885'], ['59', '2842'], ['62', '2867'], ['64', '2829'], ['63', '2844'], ['65', '2809']]]
